In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.preprocessing import StandardScaler

# Step 1: Load the dataset

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
print("First 5 rows of the dataset:")

In [ ]:
print(df.head())

In [ ]:
print("\nInformation about the dataset:")

In [ ]:
print(df.info())

# Step 2: Preprocess the data (scaling)

In [ ]:
numerical_cols = ['Retweets', 'Likes', 'Year', 'Month', 'Day', 'Hour']

In [ ]:
missing_cols = [col for col in numerical_cols if col not in df.columns]
if missing_cols:
    print(f"Warning: The following numerical columns were not found in the dataset: {missing_cols}")
    # Filter to only include columns that are actually in the DataFrame
    numerical_cols = [col for col in numerical_cols if col in df.columns]


NameError: name 'missing_cols' is not defined

In [ ]:
if not numerical_cols:
    raise ValueError("No suitable numerical columns found for clustering after initial selection. Please check the dataset columns.")

In [ ]:
X = df[numerical_cols]

In [ ]:
scaler = StandardScaler()

In [ ]:
X_scaled = scaler.fit_transform(X)

In [ ]:
print("\nData scaled successfully.")

# Step 3: Apply K-Means clustering and determine optimal K (Elbow Method)

In [ ]:
inertia = []
range_k = range(1, 11)

In [ ]:
for k in range_k:
    # Initialize KMeans with n_init to suppress warning
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range_k, inertia, marker='o', linestyle='--')
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia (Within-Cluster Sum of Squares)')
plt.grid(True)
plt.show()
print("\nElbow Method plot displayed. Look for the 'elbow' point to determine optimal K.")

# Step 4: Apply K-Means clustering with the chosen optimal K

In [ ]:
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

In [ ]:
df['Cluster'] = clusters
print(f"\nK-Means clustering applied with K={optimal_k}. Cluster assignments added to DataFrame.")

# Step 5: Visualize clusters using 2D scatter plots

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

In [ ]:
pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
pca_df['Cluster'] = clusters

In [ ]:
if 'Sentiment' in df.columns:
    pca_df['Sentiment'] = df['Sentiment']

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x='PC1',
    y='PC2',
    hue='Cluster',
    palette='viridis', # A color palette for clusters
    data=pca_df,
    legend='full',
    alpha=0.7
)

In [ ]:
plt.title(f'K-Means Clusters (PCA-Reduced Data) with K={optimal_k}')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.grid(True)
plt.show()
print("\n2D Scatter plot of clusters displayed.")

# Step 6: Interpret the clustering results

In [ ]:
cluster_analysis = df.groupby('Cluster')[numerical_cols].mean()
print("\nMean values of numerical features per cluster:")
print(cluster_analysis)

In [ ]:
if 'Sentiment' in df.columns:
    sentiment_distribution = df.groupby('Cluster')['Sentiment'].value_counts(normalize=True).unstack(fill_value=0)
    print("\nSentiment distribution (normalized) within each cluster:")
    print(sentiment_distribution)

In [ ]:
print("\nClustering analysis complete. Review the mean feature values and sentiment distribution for interpretation.")